In [92]:
from py2neo import Node, Relationship, Graph, NodeMatcher, RelationshipMatcher

In [40]:
graph = Graph('bolt://localhost:7687', name='neo4j', password='123')

In [13]:
# schema 查看與建立 graph 的 框架
# 對創建的空間下限制
graph.schema.create_uniqueness_constraint('Person', 'name')

print(graph.schema.relationship_types)
print(graph.schema.node_labels)

frozenset()
frozenset({'Person'})


In [ ]:
CREATE CONSTRAINT ON (c:Person) ASSERT c.name IS UNIQUE 
// --- 舊版寫法 (即將捨棄)
CREATE CONSTRAINT FOR (m:Movie) REQUIRE m.title IS UNIQUE 
//---- 新版寫法

## transactions & create
這兩個目的都是將現存在 python 的 點或子圖 與資料庫同步


而 transcation 像是一步一步來，而 create 是吃子圖，一步到位

In [54]:
a = Node("Person",name="Alisa",height=166)
b = Node("Person",name="Emok")
ab = Relationship(a, 'KNOWS', b)
s = a | b | ab
s

Subgraph({Node('Person', height=166, name='Alisa'), Node('Person', name='Emok')}, {KNOWS(Node('Person', height=166, name='Alisa'), Node('Person', name='Emok'))})

In [55]:
# tx 作法
# tx = graph.begin()
# tx.create(s)
# tx.commit() <---- 舊版
# graph.commit(tx) <---- 新版
# ---------
# 或等價於
graph.create(s)

通常將圖中的所有節點和關係構成一個子圖後再統一寫入資料庫，與多次寫入單個節點相比效率更高

**寫進去的點跟關係 就不會存在 python 中，所以重複執行前記得再去創建一次點**

害我搞了老半天找不到問題

## 建立 demo 圖
以 movie 為例

In [ ]:
// Nodes
CREATE (TheMatrix:Movie {title:'The Matrix', released:1999, tagline:'Welcome to the Real World'})
CREATE (Keanu:Person {name:'Keanu Reeves', born:1964})
CREATE (Carrie:Person {name:'Carrie-Anne Moss', born:1967})
CREATE (Laurence:Person {name:'Laurence Fishburne', born:1961})
CREATE (Hugo:Person {name:'Hugo Weaving', born:1960})
CREATE (LillyW:Person {name:'Lilly Wachowski', born:1967})
CREATE (LanaW:Person {name:'Lana Wachowski', born:1965})
CREATE (JoelS:Person {name:'Joel Silver', born:1952})
CREATE (Emil:Person {name:"Emil Eifrem", born:1978})
CREATE (Emil)-[:ACTED_IN {roles:["Emil"]}]->(TheMatrix)

// Relationships
CREATE
  (Keanu)-[:ACTED_IN {roles:['Neo']}]->(TheMatrix),
  (Carrie)-[:ACTED_IN {roles:['Trinity']}]->(TheMatrix),
  (Laurence)-[:ACTED_IN {roles:['Morpheus']}]->(TheMatrix),
  (Hugo)-[:ACTED_IN {roles:['Agent Smith']}]->(TheMatrix),
  (LillyW)-[:DIRECTED]->(TheMatrix),
  (LanaW)-[:DIRECTED]->(TheMatrix),
  (JoelS)-[:PRODUCED]->(TheMatrix)

In [56]:
TheMatrix = Node("Movie", title='The Matrix', released=1999, tagline='Welcome to the Real World')
Keanu = Node("Person", name='Keanu Reeves', born=1964)
Carrie = Node("Person", name='Carrie-Anne Moss', born=1967)
Laurence = Node("Person", name='Laurence Fishburne', born=1961)
Hugo = Node("Person", name='Hugo Weaving', born=1960)
LillyW = Node("Person", name='Lilly Wachowski', born=1967)
LanaW = Node("Person", name='Lana Wachowski', born=1965)
JoelS = Node("Person", name='Joel Silver', born=1952)
Emil = Node("Person", name="Emil Eifrem", born=1978)

# Relationships
LillyWTheMatrix = Relationship(LillyW, "DIRECTED", TheMatrix)
LanaWTheMatrix = Relationship(LanaW, "DIRECTED", TheMatrix)
JoelSTheMatrix = Relationship(JoelS, "PRODUCED", TheMatrix)

# 建立 關係的屬性如字典的添加一樣
# 可以直接賦予新的 key
KeanuTheMatrix = Relationship(Keanu, "ACTED_IN", TheMatrix)
KeanuTheMatrix['roles'] = ['Neo']

# 也可以將現有的字典加上去
relation_prob = {'roles': 'Trinity'}
CarrieTheMatrix = Relationship(Carrie, "ACTED_IN", TheMatrix, **relation_prob)

LaurenceTheMatrix = Relationship(Laurence, "ACTED_IN", TheMatrix)
LaurenceTheMatrix['roles'] = ['Morpheus']
HugoTheMatrix = Relationship(Hugo, "ACTED_IN", TheMatrix)
HugoTheMatrix['roles'] = ['Agent Smith']
EmilTheMatrix = Relationship(Emil, "ACTED_IN", TheMatrix)
EmilTheMatrix['roles'] = ['Emil']

In [57]:
tx = graph.begin()
tx.create(TheMatrix)
tx.create(Keanu)
tx.create(Carrie)
tx.create(Laurence)
tx.create(Hugo)
tx.create(LillyW)
tx.create(LanaW)
tx.create(JoelS)
tx.create(Emil)
tx.create(KeanuTheMatrix)
tx.create(CarrieTheMatrix)
tx.create(LaurenceTheMatrix)
tx.create(HugoTheMatrix)
tx.create(LillyWTheMatrix)
tx.create(LanaWTheMatrix)
tx.create(JoelSTheMatrix)
tx.create(EmilTheMatrix)
graph.commit(tx)

剩下的就不打 neo4j 的 demo code 有

## 搜尋

In [60]:
nodes_matcher = NodeMatcher(graph)
nodes_matcher.match()

# this is the same as
graph.nodes.match()

In [62]:
# match 出來的結果會是一個可迭代物件
# first 取出第一個
# 或是 graph.nodes.get(1)
match_using_matcher = nodes_matcher.match(name="Keanu Reeves").first()
match_using_graphnodes = graph.nodes.match(name="Keanu Reeves").first()
match_using_matcher == match_using_graphnodes

True

In [89]:
# 可下一些簡單的限制 ex: Starts with, Ends with, Less than or equal
cursor = graph.nodes.match('Movie').where("_.title CONTAINS 'Matrix' AND NOT _.title ENDS WITH 'Reloaded' ")
for node in cursor:
    print(node)

(_35:Movie {released: 1999, tagline: 'Welcome to the Real World', title: 'The Matrix'})
(_45:Movie {released: 2003, tagline: 'Everything that has a beginning has an end', title: 'The Matrix Revolutions'})


In [91]:
# MATCH (people:Person) RETURN people.name LIMIT 10
nodes_matcher.match("Person").limit(10).all()

[Node('Person', born=1964, name='Keanu Reeves'),
 Node('Person', born=1967, name='Carrie-Anne Moss'),
 Node('Person', born=1961, name='Laurence Fishburne'),
 Node('Person', born=1960, name='Hugo Weaving'),
 Node('Person', born=1967, name='Lilly Wachowski'),
 Node('Person', born=1965, name='Lana Wachowski'),
 Node('Person', born=1952, name='Joel Silver'),
 Node('Person', born=1978, name='Emil Eifrem'),
 Node('Person', born=1975, name='Charlize Theron'),
 Node('Person', born=1940, name='Al Pacino')]

## Query

In [93]:
relationship_matcher = RelationshipMatcher(graph)
relationship_matcher.match()

# this is the same as:

graph.relationships.match()

In [103]:
tom = graph.nodes.match(name="Tom Hanks").first()
all_movies = graph.match(nodes=[tom], r_type="ACTED_IN").all()
all_movies[0].end_node['title']

'A League of Their Own'

## Cypher Shell

In [104]:
results = graph.run('MATCH (m {title: "Cloud Atlas"})<-[:DIRECTED]-(directors) RETURN directors.name')
# 由 cypher 所回傳的結果稱為 cursor ，可以透過 forward 步遍
results.data()


[{'directors.name': 'Tom Tykwer'},
 {'directors.name': 'Lilly Wachowski'},
 {'directors.name': 'Lana Wachowski'}]

In [105]:
cloudAtlas = nodes_matcher.match(title="Cloud Atlas").first()
directors = graph.match(r_type="DIRECTED", nodes=(None, cloudAtlas))
for director in directors:
     print(director.nodes[0]['name'])

Lilly Wachowski
Tom Tykwer
Lana Wachowski


In [108]:
result = graph.run('MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors) RETURN coActors.name')
while result.forward():
    print(result.current)

'Madonna'
'Lori Petty'
"Rosie O'Donnell"
'Geena Davis'
'Bill Paxton'
'Philip Seymour Hoffman'
'Julia Roberts'
'Audrey Tautou'
'Ian McKellen'
'Paul Bettany'
'Jim Broadbent'
'Halle Berry'
'Hugo Weaving'
'Bonnie Hunt'
'Sam Rockwell'
'David Morse'
'James Cromwell'
'Patricia Clarkson'
'Gary Sinise'
'Michael Clarke Duncan'
'Kevin Bacon'
'Ed Harris'
'Bill Paxton'
'Gary Sinise'
'Helen Hunt'
'Dave Chappelle'
'Greg Kinnear'
'Steve Zahn'
'Parker Posey'
'Meg Ryan'
'Charlize Theron'
'Liv Tyler'
'Meg Ryan'
'Nathan Lane'
'Rita Wilson'
'Meg Ryan'
"Rosie O'Donnell"
'Victor Garber'
'Bill Pullman'


In [112]:
results = graph.run('MATCH p=shortestPath((bacon:Person {name:"Kevin Bacon"})-[*]-(meg:Person {name:"Meg Ryan"})) RETURN p')
print(results.data())
print(len(results[0]))

[{'p': Path(Node('Person', born=1958, name='Kevin Bacon'), ACTED_IN(Node('Person', born=1958, name='Kevin Bacon'), Node('Movie', released=1995, tagline='Houston, we have a problem.', title='Apollo 13'), roles=['Jack Swigert']), ACTED_IN(Node('Person', born=1956, name='Tom Hanks'), Node('Movie', released=1995, tagline='Houston, we have a problem.', title='Apollo 13'), roles=['Jim Lovell']), ACTED_IN(Node('Person', born=1956, name='Tom Hanks'), Node('Movie', released=1990, tagline='A story of love, lava and burning desire.', title='Joe Versus the Volcano'), roles=['Joe Banks']), ACTED_IN(Node('Person', born=1961, name='Meg Ryan'), Node('Movie', released=1990, tagline='A story of love, lava and burning desire.', title='Joe Versus the Volcano'), roles=['DeDe', 'Angelica Graynamore', 'Patricia Graynamore']))}]
4


4